<a href="https://colab.research.google.com/github/dsivov/MyColabs/blob/main/bert-information-retrieval/bert-weaviate-information-retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers
!pip3 install nltk
!pip3 install torch
!pip3 install weaviate-client==3.2.2

## Import the BERT transformer model and pytorch

We are using the `bert-base-uncased` model in this example, but any model will work. Feel free to adjust accordingly.

## Initialize Weaviate Client
This assumes you have Weaviate running locally on `:8080`. Adjust URL accordingly. You could also enter the WCS URL here, for example, if you are running a WCS cloud instance instead of running Weaviate locally.

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from nltk.tokenize import sent_tokenize
import weaviate

torch.set_grad_enabled(False)

# udpated to use different model if desired
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
model = AutoModel.from_pretrained(MODEL_NAME)
model.to('cuda') # remove if working without GPUs
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# initialize nltk (for tokenizing sentences)
import nltk
nltk.download('punkt')

# initialize weaviate client for importing and searching
client = weaviate.Client("http://64.71.146.93:8080")

## Load dataset from disk
Create some helper functions to create the dataset (20-newsgroup text posts) from disk. These methods are specific to the structure of your dataset, adjust accordingly.

In [ ]:
import os
import random

def get_questions_filenames(limit_objects=100):
    file_names = []
    # i=0
    # for root, dirs, files in os.walk("./data/20news-bydate-test"):
    #     for filename in files:
    #         path = os.path.join(root, filename)
    #         file_names += [path]
        
    # random.shuffle(file_names)
    # limit_objects = min(len(file_names), limit_objects)
      
    # file_names = file_names[:limit_objects]

    return file_names

def read_questions(filenames=[]):
    posts = []
    # for filename in filenames:
    #     f = open(filename, encoding="utf-8", errors='ignore')
    #     post = f.read()
        
    #     # strip the headers (the first occurrence of two newlines)
    #     post = post[post.find('\n\n'):]
        
    #     # remove posts with less than 10 words to remove some of the noise
    #     if len(post.split(' ')) < 10:
    #            continue
        
    #     post = post.replace('\n', ' ').replace('\t', ' ')
    #     if len(post) > 1000:
    #         post = post[:1000]
    #     posts += [post]

    return posts       


## Vectorize Dataset using BERT

The following is a helper function to vectorize all posts (using our BERT transformer) which are entered as an array. The return array contains all the vectors in the same order. BERT is optimized to run on GPUs, if you're using CPUs this might take a while. 

In [ ]:
import time

def text2vec(text):
    tokens_pt = tokenizer(text, padding=True, truncation=True, max_length=500, add_special_tokens = True, return_tensors="pt")
    outputs = model(**tokens_pt)
    tokens_pt.to('cuda') # remove if working without GPUs
    return outputs[0].mean(0).mean(0).detach()

def vectorize_questions(posts=[]):
    post_vectors=[]
    before=time.time()
    for i, post in enumerate(posts):
        vec=text2vec(sent_tokenize(post))
        post_vectors += [vec]
        if i % 25 == 0 and i != 0:
            print("So far {} objects vectorized in {}s".format(i, time.time()-before))
    after=time.time()
    
    print("Vectorized {} items in {}s".format(len(posts), after-before))
    
    return post_vectors

### Run everything we have so far

It is now time to run the functions we defined before. Let's load 50 random posts from disk, then vectorize them using BERT.

## Initialize Weaviate

Now that we have vectors we can import both the posts and the vectors into Weaviate, so we can then search through them.

### Init a simple schema
Our schema is very simple, we just have one object class, the "Post". A post class has just a single property, which we call "content" and is of type "text".

Each class in schema creates one index, so by running the below we tell weaviate to create one brand new vector index waiting for us to import data.

In [21]:
import json
schema = client.schema.get()
print(json.dumps(schema, indent=4))
class_obj = {
"class": "SQL_Questions", # <= Change to your class name - it will be your collection
"description": "SQL to Text mapping",
"vectorizer": "none",
"properties": [
    {
        "dataType": [
            "string"
        ],
        "description": "question",
        "name": "question"
    },
    {
        "dataType": [
            "string"
        ],
        "description": "SQL",
        "name": "SQL"
    }
]
}
client.schema.delete_class("SQL_Questions")
client.schema.create_class(class_obj)

{
    "classes": [
        {
            "class": "Weak_Sim_Intent",
            "description": "Add weak intent similarity",
            "invertedIndexConfig": {
                "bm25": {
                    "b": 0.75,
                    "k1": 1.2
                },
                "cleanupIntervalSeconds": 60,
                "stopwords": {
                    "additions": null,
                    "preset": "en",
                    "removals": null
                }
            },
            "properties": [
                {
                    "dataType": [
                        "int"
                    ],
                    "description": "Frame_nmb",
                    "name": "frame_nmbr"
                },
                {
                    "dataType": [
                        "string"
                    ],
                    "description": "movie_id",
                    "name": "movie_id",
                    "tokenization": "word"
                },
           

In [ ]:

def import_questions_with_vectors(questions, sqls, vectors, client):
    if len(questions) != len(vectors):
        raise Exception("len of posts ({}) and vectors ({}) does not match".format(len(questions), len(vectors)))
        
    for i, question in enumerate(questions):
        try:
           client.data_object.create(
                data_object={"question": question, "SQL": sqls[i]},
                class_name='SQLQuestions',
                vector=vectors[i]
            )
        except:
            print("Error")

In [ ]:
def search(query="", limit=3):
    before = time.time()
    vec = text2vec(query)
    vec_took = time.time() - before

    before = time.time()
    near_vec = {"vector": vec}
    res = client \
        .query.get("Post", ["content", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()
    search_took = time.time() - before

    print("\nQuery \"{}\" with {} results took {:.3f}s ({:.3f}s to vectorize and {:.3f}s to search)" \
          .format(query, limit, vec_took+search_took, vec_took, search_took))
    for post in res["data"]["Get"]["Post"]:
        print("{:.4f}: {}".format(post["_additional"]["certainty"], post["content"]))
        print('---')

In [ ]:
questions = read_questions(get_questions_filenames(100))
vectors = vectorize_questions(questions)
import_questions_with_vectors(questions, vectors, client)

In [ ]:
search("the best camera lens", 1)
search("motorcycle trip", 1)
search("which software do i need to view jpeg files", 1)
search("windows vs mac", 1)

In [ ]:
#
